In [260]:
import pandas as pd
from nltk.tokenize import TreebankWordTokenizer

from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import spacy

In [261]:
df_raw = pd.read_pickle('df_raw.pickle')

In [4]:
#turn to csv so we can store locally and store as database in sql
df_raw.to_csv('/Users/juliaqiao/Documents/Metis/Proj_4_storage/df_raw.csv')

### Preprocessing

In [102]:
df_raw.columns

Index(['user', 'date', 'outlinks', 'content', 'url'], dtype='object')

In [262]:
#reindex the columns for easier viewing
cols = df_raw.columns.tolist()

cols.insert(2, cols.pop(cols.index('url')))

df_raw= df_raw.reindex(columns= cols)


In [14]:
# get dictionary value of key: username
# handle = [d[0].get('username') for d in df_raw.user if d]

In [263]:
def preprocess(tweet):
    """
    Takes in tweet and performs initial text cleaning/preprocessing.
    """
    #make sure doc is string
    tweet=str(tweet)
    #lowercase-- not changing anything to lowercase yet due to proper nouns being very important. want to use Spacy to detect later.
    #tweet = tweet.lower()
    #get rid of urls
    rem_url=re.sub(r'http\S+', '', tweet)
    #gets rid of @ tags
    rem_tag = re.sub('@\S+', '', rem_url)
    #gets rid of # in hashtag but keeps content of hashtag
    rem_hashtag = re.sub('#', '', rem_tag)
    #gets rid of numbers
    rem_num = re.sub('[0-9]+', '', rem_hashtag)
    #gets rid of special characters
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', rem_num)

    return cleantext

In [264]:
df_raw['clean']=df_raw['content'].map(lambda x:preprocess(x))

### Tokenize

In [265]:
tt = TreebankWordTokenizer()
df_raw['clean'] = df_raw['clean'].apply(tt.tokenize)

In [266]:
df_raw['clean'].iloc[-1]

['The',
 'coronavirus',
 'pandemic',
 'is',
 'devastating',
 'U.S.',
 'restaurant',
 'chains.',
 'But',
 'explains',
 'why',
 'financially',
 'stronger',
 'companies',
 'should',
 'see',
 'the',
 'light',
 'of',
 'day',
 'in',
 'the',
 'near',
 'future.',
 'WSJWhatsNow']

 ### Lemmatize 

In [267]:
lemmatizer = WordNetLemmatizer()
df_raw['clean'] = df_raw['clean'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x]) # Lemmatize every word.

In [203]:
#run example
df_raw['clean'].iloc[-1]

['The',
 'coronavirus',
 'pandemic',
 'is',
 'devastating',
 'U.S.',
 'restaurant',
 'chains.',
 'But',
 'explains',
 'why',
 'financially',
 'stronger',
 'company',
 'should',
 'see',
 'the',
 'light',
 'of',
 'day',
 'in',
 'the',
 'near',
 'future.',
 'WSJWhatsNow']

### Remove Stop Words

In [268]:
#removes default stop_words
stop_words = list(stopwords.words('english'))

In [269]:
#stop_words = set(stopwords.words('english'))
df_raw['clean'] = df_raw['clean'].apply(lambda x: [word for word in x if word not in stop_words])

In [207]:
#run example
df_raw['clean'].iloc[-1]

['The',
 'coronavirus',
 'pandemic',
 'devastating',
 'U.S.',
 'restaurant',
 'chains.',
 'But',
 'explains',
 'financially',
 'stronger',
 'company',
 'see',
 'light',
 'day',
 'near',
 'future.',
 'WSJWhatsNow']

We did not use a stemmer. Lemmatizer was better.

Remove any leftover punctuation:


In [270]:
punctuation = ['.', '/', ',', '%', '!', ':', "'s", "'ve", '"', '*', '“', "’", "`", '?', '”', "``", "‘"] 
df_raw['clean'] = df_raw['clean'].apply(lambda x: [word for word in x if word not in punctuation])

In [271]:
df_raw['clean'].sample(20)

3977     [Unfinished, puzzle, half-done, scarf, unopene...
4883     [This, wa, decade, American, population, turne...
9346     [In, latest, issue, Fiona, Apple, discus, conf...
17766    [T., S., Eliot, helped, make, Faber, &, amp, ;...
13586    [Around, world, nationalist, government, claim...
16680    [Physical, books—which, ten, year, ago, many, ...
15779    [At, least, million, move, around, again., Pub...
13939    [Twitter, temporarily, blocked, Trump, campaig...
2410      [specific, way, tech, address, systemic, racism]
14063    [Planet, Fitness, ', revenue, fell, Q, attribu...
5006     [Scores, civilian, still, killed, wounded, eve...
14154    [The, overlap, Asia, though, minimal, give, Eu...
18849    [This, year, celebrate, th, birthday, Walt, Wh...
17040    [LIVE, Attorney, General, Barr, hold, press, c...
2472     [The, chain, flimsy, commitment, reminder, man...
2034     [Every, Presidency, ha, dissenter, people, lea...
12312    [Many, supermarket, remain, skeptical, deliver.

### Remove any additional words like pronouns and other fillers

In [279]:
pronoun_words = ['The', "the", "I", "he", "He", "She", "she", "her", "Her", "We", "we", "Me", "me", "Us", "us", "it","It", "Them", "them", "They", "they", "There", "there", "that", "That", "This", "this", "You", "you", "who", "Who", "whom" , "Whom", "whose", "Whose", "what", "What", "which", "Which", "my", "My", "mine","Mine", "your", "Your", "yours", "Yours", "his", "His", "hers" "Hers", "its", "Its", "It's", "it's", "Is", "is" "was", "Was", "In", "in", "across", "Across", "rather", "Rather", "roughly", "Roughly", "why", "Why", "where", "Where", "here", "Here", "A", "a", "Some", "some", "few", "Few", "None", "none", "more", "More", "Yes", "yes", "No", "no", "If", "if", "Let", "let", "Let's", "let's", "Came", "come", "go", "Go"]

In [280]:
#stop_words = set(stopwords.words('english'))
df_raw['clean'] = df_raw['clean'].apply(lambda x: [word for word in x if word not in pronoun_words])

In [281]:
df_raw['clean'].sample(20)

14143    [Donald, Trump, ha, signed, legislation, new, ...
5065                                   [Theodore, trouble]
5372     [Coronavirus, update, Infections, rise, Univer...
15058    [insertion, state, physician, office, —, priva...
1713     [General, Motors, ', Cruise, pandemic-proof, d...
18722    [From, ice-cream, tin, food, wrap, made, seawe...
5977     [coronavirus, lockdown, forced, many, people, ...
14601    [half, year, since, last, Red, Ed, strike, end...
17715    [Top, official, Brussels, believe, UK, populat...
3278               [Worrying, people, behavior, n't, good]
11126    [beauty, ha, floor-to-ceiling, window, take, S...
10787    [latest, coronavirus, news, 👉NY, attorney-gene...
16228    [Milagros, baby, Cristal, wa, born, premature,...
13404    [Florida-sized, hunk, frozen, water, Antarctic...
7014     [Los, Angeles, businessman, tip, led, governme...
19064    [Perspective, massive, wave, eviction, coming....
11398    [new, science, synthetic, biology, giving, u, .

### Vectorize

In [282]:
df_raw['clean_vec'] = df_raw['clean'].apply(lambda x: ' '.join(x))


In [283]:
df_raw['clean_vec']

0                           Atlantic Daily Will decade new
1        plenty going wrong Trump. four thing campaign ...
2        Trump try steal election people surely protest...
3        Trump campaign election-security operation key...
4        Even Joe Biden win decisively next week coming...
                               ...                        
19995    General Mills make Cheerios cereal Yoplait yog...
19996    From When viral panic subsides Fed rethink mor...
19997           government want send money—but soon arrive
19998    frozen asparagus said one disappointed shopper...
19999    coronavirus pandemic devastating U.S. restaura...
Name: clean_vec, Length: 200000, dtype: object

In [284]:
tfidf = TfidfVectorizer()


In [285]:
doc_term_matrix = tfidf.fit_transform(df_raw['clean_vec'])

In [286]:
df = pd.DataFrame(doc_term_matrix.toarray(), columns=tfidf.get_feature_names())

In [287]:
df.head()

,__,___,____,_____,______________,_______________________,__________________________,__zz,_zz,aa,...,ﬂora,ﬂow,ﬂuﬀy,ﬂy,ﬂygskam,ﬂying,𝐀𝐯𝐚𝐢𝐥𝐚𝐛𝐥𝐞,𝐔𝐩𝐝𝐚𝐭𝐞𝐬,𝘧𝘰𝘳,𝘩𝘪𝘮
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [291]:
df.columns[40000:41000]

Index(['propellant', 'propelled', 'propeller', 'propelling', 'propels',
       'propensity', 'proper', 'properly', 'properties', 'property',
       ...
       'raised', 'raiser', 'raisers', 'raises', 'raisin', 'raising', 'raison',
       'raj', 'raja', 'rajabian'],
      dtype='object', length=1000)

In [293]:
df.to_pickle("doc_term_matrix_0.pickle")

### Scratch

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

class DenseTfidfVectorizer(TfidfVectorizer):

    def transform(self, raw_documents, copy=True):
        X = super().transform(raw_documents, copy=copy)
        df = pd.DataFrame(X.toarray(), columns=self.get_feature_names())
        return df

    def fit_transform(self, raw_documents, y=None):
        X = super().fit_transform(raw_documents, y=y)
        df = pd.DataFrame(X.toarray(), columns=self.get_feature_names())
        return df